# Embedding and VectorDB

- [Welcome to Faiss Documentation — Faiss documentation](https://faiss.ai/index.html)

## Data Preparation

### URL parsing (deprecated)

In [12]:
import os
from typing import List
import urllib.request
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [39]:
from dotenv import load_dotenv
load_dotenv(os.path.join(os.getcwd(), '../.env'))

True

In [8]:
URLS = {
    "樹木會鼓掌": "https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6",
    "剩食革命": "https://learning.wisdomhall.com.tw/posts/63992d65fea0170039ac236a",
    "不一樣的銀行": "https://learning.wisdomhall.com.tw/posts/646efba3b812e200398c8a75",
}

In [9]:
local_file_path = os.path.join(os.getcwd(), "data")
os.makedirs(local_file_path, exist_ok=True)
for name, url in URLS.items():
    destination_path = os.path.join(local_file_path, name)
    urllib.request.urlretrieve(url, destination_path)

In [10]:
def get_file_chunks(file_name: str) -> List[str]:
    with open(file_name, "r", encoding="utf-8") as f:
        page_content = f.read()
        # use BeautifulSoup to parse HTML content
        soup = BeautifulSoup(page_content, "html.parser")
        text = soup.get_text(" ", strip=True)
        chunks = []
        splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=10)
        for chunk in splitter.split_text(text):
            chunks.append(chunk)
        return chunks

In [11]:
for name in URLS:
    file_name = os.path.join(local_file_path, name)
    chunks = get_file_chunks(file_name)
    count = len(chunks)
    print(file_name, count, chunks)

C:\Users\david\Documents\Program\PromptFlow\notebooks\data\樹木會鼓掌 2 ['品學堂閱讀理解數位學習系統｜樹木會鼓掌 我要訂閱 教育部111年「校園數位內容與教學軟體」第一次選購名單 產品流水號1031 | 產品序號11112-085 首頁 文章列表 訂閱方案 最新消息 關於我們 操作指南 閱讀理解 junior 我的訂閱 我的學習 我的資料 我的行事曆 我的收藏 登出 PISA閱讀素養線上評量 我要訂閱 登入 註冊 取消 登入 註冊 樹木會鼓掌 本文內容及評量版權歸品學堂文化股份有限公司所有，任何單位及個人未經許可，不得擅自轉載。 樹木會鼓掌 ARTICLES 人物 #junior #SDGs就在你身邊 #2023 #利他 樹木會鼓掌 2023-06-01 作者： 王文華 出版社： 親子天下 #junior #SDGs就在你身邊 #2023 #利他 人物 連續 敘事型 簡單 A A © 2013-2024 品學堂文化股份有限公司 版權所有 火！火！火！乾燥的森林，冒出一場大火，火光衝天，情勢緊急。動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠罩，牠們心有餘悸，充滿了無力感，面對如此的災害，誰都拿不出辦法……「不！」有隻小蜂鳥對自己說：「我不想看著森林被燒個精光，我一定能做點什麼！」牠飛到鄰近小溪，舀水來救火，弱小的蜂鳥，即使張大了嘴，一次也只能舀起一滴水，含著它，千辛萬苦飛回火場，倒進大火之中，然後第二滴、第三滴……一隻大嘴鳥搖搖頭：「這麼大的火，這樣做有意義嗎？」小蜂鳥回答：「當然，因為我在盡己所能！」說這個故事的人名叫萬嘉麗·瑪泰(Wangari Muta Maathai)，她來自肯亞... 註冊免費看更多 作答時間： 15\n              分鐘 開始答題 © 2013-2024 品學堂文化股份有限公司 版權所有 請依據前述文章回答下列問題。 （  ）1.萬嘉麗利用小蜂鳥的故事說明什麼道理？ 1. 懂得運用經驗解決難題，小兵也可以立大功 2. 眾人齊力能斷金，強調團隊分工合作的重要 3. 要勇於面對挑戰，並且能從錯誤中不斷學習 4. 儘管個人力量微薄，仍該努力嘗試解決問題 本文內容及評量版權歸品學堂文化股份有限公司所有，任何單位及個人未經許可，不得

### Manually Defined JSON

In [16]:
import json

In [17]:
json_files = [
    "../data/wisdomhall/樹木會鼓掌.json"
]

In [21]:
with open(json_files[0], 'r', encoding='utf-8') as fp:
    article_obj = json.load(fp)
article_obj

{'title': '樹木會鼓掌',
 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6',
 'date': '2023-06-01',
 'author': '王文華',
 'publisher': '親子天下',
 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'],
 'paragraphs': ['火！火！火！',
  '乾燥的森林，冒出一場大火，火光衝天，情勢緊急。',
  '動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠罩，牠們心有餘悸，充滿了無力感，面對如此的災害，誰都拿不出辦法……',
  '「不！」有隻小蜂鳥對自己說：「我不想看著森林被燒個精光，我一定能做點什麼！」',
  '牠飛到鄰近小溪，舀水來救火，弱小的蜂鳥，即使張大了嘴，一次也只能舀起一滴水，含著它，千辛萬苦飛回火場，倒進大火之中，然後第二滴、第三滴……',
  '一隻大嘴鳥搖搖頭：「這麼大的火，這樣做有意義嗎？」',
  '小蜂鳥回答：「當然，因為我在盡己所能！」',
  '說這個故事的人名叫萬嘉麗·瑪泰(Wangari Muta Maathai)，她來自肯亞。萬嘉麗的臉上，永遠有著非洲陽光般燦爛的笑容，因為她相信：即使是最幽暗的地方，也總有一絲微光，讓人看見希望。對照她的一生，其實就像那隻小蜂鳥，盡己所能，勇往直前。',
  '萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。',
  '身為長女的萬嘉麗，經常和媽媽到田裡工作。農人的工作是辛苦的，每天早上，她必須和媽媽涉水渡過幾條河，才能抵達他們家的土地。媽媽常一邊耕種，一邊教她肯亞人口耳相傳的古老智慧，例如要尊敬無花果樹，因為它是神樹，不應該使用它，更不能砍伐它。',
  '「種點東西，勝過袖手旁觀！」或許你的媽媽也常跟你說過類似這樣「要行動、別光說不練」的話，但你是不是把它們當成嘮叨，最後什麼也沒記住呢？',
  '不過萬嘉麗不是喔，她就像非洲那片

In [26]:
article_obj.keys()

dict_keys(['title', 'url', 'date', 'author', 'publisher', 'tags', 'paragraphs', 'QA'])

In [25]:
print('\n'.join(article_obj['paragraphs']))

火！火！火！
乾燥的森林，冒出一場大火，火光衝天，情勢緊急。
動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠罩，牠們心有餘悸，充滿了無力感，面對如此的災害，誰都拿不出辦法……
「不！」有隻小蜂鳥對自己說：「我不想看著森林被燒個精光，我一定能做點什麼！」
牠飛到鄰近小溪，舀水來救火，弱小的蜂鳥，即使張大了嘴，一次也只能舀起一滴水，含著它，千辛萬苦飛回火場，倒進大火之中，然後第二滴、第三滴……
一隻大嘴鳥搖搖頭：「這麼大的火，這樣做有意義嗎？」
小蜂鳥回答：「當然，因為我在盡己所能！」
說這個故事的人名叫萬嘉麗·瑪泰(Wangari Muta Maathai)，她來自肯亞。萬嘉麗的臉上，永遠有著非洲陽光般燦爛的笑容，因為她相信：即使是最幽暗的地方，也總有一絲微光，讓人看見希望。對照她的一生，其實就像那隻小蜂鳥，盡己所能，勇往直前。
萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。
身為長女的萬嘉麗，經常和媽媽到田裡工作。農人的工作是辛苦的，每天早上，她必須和媽媽涉水渡過幾條河，才能抵達他們家的土地。媽媽常一邊耕種，一邊教她肯亞人口耳相傳的古老智慧，例如要尊敬無花果樹，因為它是神樹，不應該使用它，更不能砍伐它。
「種點東西，勝過袖手旁觀！」或許你的媽媽也常跟你說過類似這樣「要行動、別光說不練」的話，但你是不是把它們當成嘮叨，最後什麼也沒記住呢？
不過萬嘉麗不是喔，她就像非洲那片乾燥的大地，把媽媽的話當成珍貴的雨水，不但記下來，更是身體力行，而「種點東西，勝過袖手旁觀」，竟成了她一生的座右銘。
這樣的孩子人見人愛，修女們幫助她進入學校讀書，她珍惜求學的機會，在學校的表現優異，得到出國留學的獎學金，她先到美國，再到德國。她在國外的大學裡學到許多新鮮的事物，也研究植物以及它們的生長過程。
讀書很辛苦，收穫卻很豐富，而且讀到疲憊的時候，萬嘉麗就會想想遠在幾千公里外的家鄉，還有童年時，和兄弟姐妹在伊希提樹林裡遊玩的情景。
那片美麗的景色，故鄉善良親切的人們，像是在朝她頻頻招手。
學得越多，越想念家鄉，這個學有所成的女孩，只想趕快回家，

In [30]:
chunk_metadata_pairs = []
for file_name in json_files:
    with open(file_name, 'r', encoding='utf-8') as fp:
        article_obj = json.load(fp)
    metadata_temp = {
        k: v
        for k, v in article_obj.items()
        if k in {'title', 'url', 'date', 'author', 'publisher', 'tags'}
    }
    for i, p in enumerate(article_obj['paragraphs']):
        metadata = metadata_temp.copy()
        metadata['paragraph_id'] = i
        chunk_metadata_pairs.append((p, metadata))
        
chunk_metadata_pairs

[('火！火！火！',
  {'title': '樹木會鼓掌',
   'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6',
   'date': '2023-06-01',
   'author': '王文華',
   'publisher': '親子天下',
   'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'],
   'paragraph_id': 0}),
 ('乾燥的森林，冒出一場大火，火光衝天，情勢緊急。',
  {'title': '樹木會鼓掌',
   'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6',
   'date': '2023-06-01',
   'author': '王文華',
   'publisher': '親子天下',
   'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'],
   'paragraph_id': 1}),
 ('動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠罩，牠們心有餘悸，充滿了無力感，面對如此的災害，誰都拿不出辦法……',
  {'title': '樹木會鼓掌',
   'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6',
   'date': '2023-06-01',
   'author': '王文華',
   'publisher': '親子天下',
   'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'],
   'paragraph_id': 2}),
 ('「不！」有隻小蜂鳥對自己說：「我不想看著森林被燒個精光，我一定能做點什麼！」',
  {'title': '樹木會鼓掌',
   'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6',
   '

In [36]:
chunks, metadatas = list(zip(*chunk_metadata_pairs))
chunks, metadatas

(('火！火！火！',
  '乾燥的森林，冒出一場大火，火光衝天，情勢緊急。',
  '動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠罩，牠們心有餘悸，充滿了無力感，面對如此的災害，誰都拿不出辦法……',
  '「不！」有隻小蜂鳥對自己說：「我不想看著森林被燒個精光，我一定能做點什麼！」',
  '牠飛到鄰近小溪，舀水來救火，弱小的蜂鳥，即使張大了嘴，一次也只能舀起一滴水，含著它，千辛萬苦飛回火場，倒進大火之中，然後第二滴、第三滴……',
  '一隻大嘴鳥搖搖頭：「這麼大的火，這樣做有意義嗎？」',
  '小蜂鳥回答：「當然，因為我在盡己所能！」',
  '說這個故事的人名叫萬嘉麗·瑪泰(Wangari Muta Maathai)，她來自肯亞。萬嘉麗的臉上，永遠有著非洲陽光般燦爛的笑容，因為她相信：即使是最幽暗的地方，也總有一絲微光，讓人看見希望。對照她的一生，其實就像那隻小蜂鳥，盡己所能，勇往直前。',
  '萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。',
  '身為長女的萬嘉麗，經常和媽媽到田裡工作。農人的工作是辛苦的，每天早上，她必須和媽媽涉水渡過幾條河，才能抵達他們家的土地。媽媽常一邊耕種，一邊教她肯亞人口耳相傳的古老智慧，例如要尊敬無花果樹，因為它是神樹，不應該使用它，更不能砍伐它。',
  '「種點東西，勝過袖手旁觀！」或許你的媽媽也常跟你說過類似這樣「要行動、別光說不練」的話，但你是不是把它們當成嘮叨，最後什麼也沒記住呢？',
  '不過萬嘉麗不是喔，她就像非洲那片乾燥的大地，把媽媽的話當成珍貴的雨水，不但記下來，更是身體力行，而「種點東西，勝過袖手旁觀」，竟成了她一生的座右銘。',
  '這樣的孩子人見人愛，修女們幫助她進入學校讀書，她珍惜求學的機會，在學校的表現優異，得到出國留學的獎學金，她先到美國，再到德國。她在國外的大學裡學到許多新鮮的事物，也研究植物以及它們的生長過程。',
  '讀書很辛苦，收穫卻很豐富，而且讀到疲憊的時候，萬嘉麗就會想想遠在幾千公里外的家鄉，還有童年時，和兄弟姐妹在伊希

## Prompt Flow Example

- [azureml-examples/sdk/python/generative-ai/promptflow/create_faiss_index.ipynb at main · Azure/azureml-examples](https://github.com/Azure/azureml-examples/blob/main/sdk/python/generative-ai/promptflow/create_faiss_index.ipynb)

In [1]:
%pip install promptflow-vectordb

  Using cached langchain-0.0.348-py3-none-any.whl.metadata (16 kB)
  Using cached langchain_core-0.0.13-py3-none-any.whl.metadata (978 bytes)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
Using cached langchain-0.0.348-py3-none-any.whl (2.0 MB)
Using cached langchain_core-0.0.13-py3-none-any.whl (188 kB)
Using cached langsmith-0.0.92-py3-none-any.whl (56 kB)
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.1.27
    Uninstalling langsmith-0.1.27:
      Successfully uninstalled langsmith-0.1.27
  Rolling back uninstall of langsmith
  Moving to c:\users\david\appdata\roaming\python\python311\scripts\langsmith.exe
   from C:\Users\david\AppData\Local\Temp\pip-uninstall-if40h58f\langsmith.exe
  Moving to c:\users\david\appdata\roaming\python\python311\site-packages\langsmith-0.1.27.dist-info\
   from C:\Users\david\AppData\Roaming\Python\Python311\site-packages\~angsmith-0.1.27.dist-info
  Moving to c:\users\david\appdata\roaming\python\pyt

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\langsmith.exe' -> 'C:\\Python311\\Scripts\\langsmith.exe.deleteme'


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from promptflow_vectordb.core.contracts import (
    EmbeddingModelType,
    StorageType,
    StoreCoreConfig,
)
from promptflow_vectordb.core.embeddingstore_core import EmbeddingStoreCore

In [40]:
MODEL_API_VERSION = "2024-02-15-preview"
MODEL_DEPLOYMENT_NAME = "text-embedding-ada-002"
DIMENSION = 1536

# TODO: solve warning
# Configure an embedding store to store index file.
store_path = os.path.join(os.getcwd(), "faiss_index_store")
config = StoreCoreConfig.create_config(
    storage_type=StorageType.LOCAL,
    store_identifier=store_path,
    model_type=EmbeddingModelType.AOAI,
    model_api_base=os.environ["AZURE_OPENAI_ENDPOINT"],
    model_api_key=os.environ["AZURE_OPENAI_KEY"],
    model_api_version=MODEL_API_VERSION,
    model_name=MODEL_DEPLOYMENT_NAME,
    dimension=DIMENSION,
    create_if_not_exists=True,
)
store = EmbeddingStoreCore(config)
store

`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.
`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [41]:
store.batch_insert_texts(chunks, metadatas)

In [42]:
store.search_by_text("肯亞的牛")

[SearchResultEntity(text='她覺得肯亞的牛都太瘦弱了，她在別的國家看到的牛，不管是乳牛還是水牛，長得都是又肥又壯，乳牛有牛奶，水牛可耕田，回頭看看肯亞的牛，每一隻都是瘦弱不堪。別說耕田了，好像走一走，都快要倒下來。', vector=None, score=0.20859400928020477, original_entity=None, metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 18}),
 SearchResultEntity(text='萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。', vector=None, score=0.3654283881187439, original_entity=None, metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 8}),
 SearchResultEntity(text='不過萬嘉麗不是喔，她就像非洲那片乾燥的大地，把媽媽的話當成珍貴的雨水，不但記下來，更是身體力行，而「種點東西，勝過袖手旁觀」，竟成了她一生的座右銘。', vector=None, score=0.3852950

In [45]:
store.search_by_text("主角的名字")

[SearchResultEntity(text='她說的這段話，像不像文章一開始，那隻身形弱小，卻展現強大決心與行動力的小蜂鳥？', vector=None, score=0.41626954078674316, original_entity=None, metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 40}),
 SearchResultEntity(text='說這個故事的人名叫萬嘉麗·瑪泰(Wangari Muta Maathai)，她來自肯亞。萬嘉麗的臉上，永遠有著非洲陽光般燦爛的笑容，因為她相信：即使是最幽暗的地方，也總有一絲微光，讓人看見希望。對照她的一生，其實就像那隻小蜂鳥，盡己所能，勇往直前。', vector=None, score=0.4203232526779175, original_entity=None, metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 7}),
 SearchResultEntity(text='身為長女的萬嘉麗，經常和媽媽到田裡工作。農人的工作是辛苦的，每天早上，她必須和媽媽涉水渡過幾條河，才能抵達他們家的土地。媽媽常一邊耕種，一邊教她肯亞人口耳相傳的古老智慧，例如要尊敬無花果樹，因為它是神樹，不應該使用它，更不能砍伐它。', vector=None, score=0.4407702684402466, origina

## LangChain Example

- [Faiss | 🦜️🔗 Langchain](https://python.langchain.com/docs/integrations/vectorstores/faiss)
- [Azure OpenAI | 🦜️🔗 Langchain](https://python.langchain.com/docs/integrations/text_embedding/azureopenai)

In [47]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [48]:
embeddings = AzureOpenAIEmbeddings(api_key=os.environ["AZURE_OPENAI_KEY"], azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"], azure_deployment=MODEL_DEPLOYMENT_NAME, api_version=MODEL_API_VERSION, dimensions=DIMENSION)
embeddings

AzureOpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x000002189B74E650>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000021898FDB4D0>, model='text-embedding-ada-002', dimensions=1536, deployment='text-embedding-ada-002', openai_api_version='2024-02-15-preview', openai_api_base=None, openai_api_type='azure', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=16, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, azure_endpoint='https://openaihzhou.openai.azure.com/', azure_ad_token=None, azure_ad_token_provider=None, validate_base_url=True)

In [49]:
embeddings.embed_query("主角的名字")

[-0.020908352202967894,
 -0.02117778941451495,
 4.462559127697255e-05,
 -0.01639527285595788,
 -0.024774780752148833,
 0.03144335779153531,
 -0.018995344834486998,
 -0.010595630963509056,
 -0.022861773383667936,
 -0.008298675615139733,
 0.017971482313021078,
 0.0073017567216962604,
 -0.003910211886509755,
 0.020504195454324723,
 -0.014293660556981147,
 0.002408097988584903,
 0.014051166507795244,
 0.013795200877428764,
 0.020598499316556516,
 -0.02264622435948836,
 0.020625442478917674,
 -0.005867002126658173,
 -0.015101973122944904,
 0.010124115377640413,
 -0.02505769047619905,
 0.01794453915065992,
 0.027859840829049753,
 -0.014832535911397845,
 0.012791546193395002,
 -0.031039202905537303,
 0.005988249151251125,
 0.0010070226957442287,
 0.005082265526252365,
 -0.022875245896171098,
 -0.005045217513852547,
 0.009928773259570409,
 0.007348908652812158,
 -0.006230742734775734,
 0.011073881874306695,
 0.022390257797799292,
 0.004917234698669307,
 -0.0021975997596101336,
 0.0004731996499

In [53]:
# https://stackoverflow.com/questions/76551067/how-to-create-a-langchain-doc-from-an-str
from langchain.docstore.document import Document

docs = []
for chunk, metadata in chunk_metadata_pairs:
    docs.append(
        Document(page_content=chunk, metadata=metadata)
    )
docs

[Document(page_content='火！火！火！', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 0}),
 Document(page_content='乾燥的森林，冒出一場大火，火光衝天，情勢緊急。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 1}),
 Document(page_content='動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠罩，牠們心有餘悸，充滿了無力感，面對如此的災害，誰都拿不出辦法……', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 2}),
 Document(page_content='「不！」有隻小蜂鳥對自己說：「我不想看著森林被燒個精光，我一定能做點什麼！」', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.

In [54]:
faiss_index = FAISS.from_documents(docs, embeddings)
faiss_index

In [65]:
faiss_index.index.ntotal

45

In [55]:
faiss_index.similarity_search("主角的名字")

[Document(page_content='她說的這段話，像不像文章一開始，那隻身形弱小，卻展現強大決心與行動力的小蜂鳥？', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 40}),
 Document(page_content='說這個故事的人名叫萬嘉麗·瑪泰(Wangari Muta Maathai)，她來自肯亞。萬嘉麗的臉上，永遠有著非洲陽光般燦爛的笑容，因為她相信：即使是最幽暗的地方，也總有一絲微光，讓人看見希望。對照她的一生，其實就像那隻小蜂鳥，盡己所能，勇往直前。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 7}),
 Document(page_content='身為長女的萬嘉麗，經常和媽媽到田裡工作。農人的工作是辛苦的，每天早上，她必須和媽媽涉水渡過幾條河，才能抵達他們家的土地。媽媽常一邊耕種，一邊教她肯亞人口耳相傳的古老智慧，例如要尊敬無花果樹，因為它是神樹，不應該使用它，更不能砍伐它。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['j

In [64]:
faiss_index.similarity_search("肯亞的牛", k=1, fetch_k=4)

[Document(page_content='她覺得肯亞的牛都太瘦弱了，她在別的國家看到的牛，不管是乳牛還是水牛，長得都是又肥又壯，乳牛有牛奶，水牛可耕田，回頭看看肯亞的牛，每一隻都是瘦弱不堪。別說耕田了，好像走一走，都快要倒下來。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 18})]

In [57]:
retriever = faiss_index.as_retriever()
retriever.invoke("肯亞的牛")

[Document(page_content='她覺得肯亞的牛都太瘦弱了，她在別的國家看到的牛，不管是乳牛還是水牛，長得都是又肥又壯，乳牛有牛奶，水牛可耕田，回頭看看肯亞的牛，每一隻都是瘦弱不堪。別說耕田了，好像走一走，都快要倒下來。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 18}),
 Document(page_content='萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 8}),
 Document(page_content='不過萬嘉麗不是喔，她就像非洲那片乾燥的大地，把媽媽的話當成珍貴的雨水，不但記下來，更是身體力行，而「種點東西，勝過袖手旁觀」，竟成了她一生的座右銘。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '

In [58]:
faiss_index.similarity_search_with_score("有幾個兄弟姊妹")

[(Document(page_content='讀書很辛苦，收穫卻很豐富，而且讀到疲憊的時候，萬嘉麗就會想想遠在幾千公里外的家鄉，還有童年時，和兄弟姐妹在伊希提樹林裡遊玩的情景。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 13}),
  0.36621022),
 (Document(page_content='萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 8}),
  0.36929756),
 (Document(page_content='那片美麗的景色，故鄉善良親切的人們，像是在朝她頻頻招手。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2

In [59]:
faiss_index.save_local("langchain_faiss_index")

In [60]:
faiss_index.docstore._dict

{'67753817-18c8-4c94-8516-fd9cc55d8898': Document(page_content='火！火！火！', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 0}),
 '7c2e989a-6779-41a8-995b-0782bd08fb6d': Document(page_content='乾燥的森林，冒出一場大火，火光衝天，情勢緊急。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 1}),
 '1eb370be-8270-43a7-b0e1-f54e4bed86ea': Document(page_content='動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠罩，牠們心有餘悸，充滿了無力感，面對如此的災害，誰都拿不出辦法……', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 2})

In [62]:
faiss_index.similarity_search_with_score("肯亞的牛", filter=lambda d: d["paragraph_id"] < 10)

[(Document(page_content='萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 8}),
  0.3654484),
 (Document(page_content='身為長女的萬嘉麗，經常和媽媽到田裡工作。農人的工作是辛苦的，每天早上，她必須和媽媽涉水渡過幾條河，才能抵達他們家的土地。媽媽常一邊耕種，一邊教她肯亞人口耳相傳的古老智慧，例如要尊敬無花果樹，因為它是神樹，不應該使用它，更不能砍伐它。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 9}),
  0.39489642),
 (Document(page_content='說這個故事的人名叫萬嘉麗·瑪泰(Wangari Muta Maathai)，她來自肯亞。萬嘉麗的臉上，永遠有著非洲陽光般燦爛的笑容，因為她相信：即使是最幽暗的地方，也總有一絲微光，讓人看見希望。對照她的一生，其實就像那隻小蜂鳥，盡己所能，勇往直前。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.

In [63]:
faiss_index.max_marginal_relevance_search("肯亞的牛", filter=lambda d: d["paragraph_id"] < 10)

[Document(page_content='萬嘉麗出生於非洲肯亞的伊希提村落，她是長女，上有兩個哥哥，下有三個弟妹。她的父母都是農人，在她出生的時代，伊希提是個林木青綠、土壤肥沃的好地方，那時季節變化正常，雨量豐沛。農田裡種植著玉蜀黍、豌豆、小麥和蔬菜，只要願意勞動，田裡的收穫足以讓他們一家人溫飽無虞。', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 8}),
 Document(page_content='火！火！火！', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 0}),
 Document(page_content='小蜂鳥回答：「當然，因為我在盡己所能！」', metadata={'title': '樹木會鼓掌', 'url': 'https://learning.wisdomhall.com.tw/posts/646f07fab812e20039962bb6', 'date': '2023-06-01', 'author': '王文華', 'publisher': '親子天下', 'tags': ['junior', 'SDGs就在你身邊', '2023', '利他'], 'paragraph_id': 6}),
 Document(page_content='動物們拚命的往外逃，好不容易來到安全的地方，回首遙望原本翠綠的林子，現在已被烈焰濃煙籠